In [ ]:
#Célula 1: Instalação de bibliotecas adicionais (se necessário)
!pip install category_encoders xgboost

In [ ]:
#Célula 2: Importações
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from shapash import SmartExplainer

#Configuração de visualização
pd.set_option('display.max_columns', None)

In [ ]:
#Célula 3: Carregamento e Preparação Rápida (Recapitulando Notebook 01)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
df = pd.read_csv(url)

#Renomeando para Português (Foco no Operador - Human Centricity)
mapa_nomes = {
    'Type': 'Tipo_Equipamento',
    'Air temperature [K]': 'Temp_Ar',
    'Process temperature [K]': 'Temp_Processo',
    'Rotational speed [rpm]': 'Rotacao_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Desgaste_Ferramenta_min',
    'Machine failure': 'Falha'
}
df = df.rename(columns=mapa_nomes)

#Removendo colunas desnecessárias e data leakage
df_model = df.drop(['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

# Separando X (Features) e y (Target)
X = df_model.drop('Falha', axis=1)
y = df_model['Falha']

In [ ]:
#Célula 4: Divisão Treino e Teste

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
#Célula 5: Codificação de Variáveis Categóricas

encoder = OrdinalEncoder(cols=['Tipo_Equipamento'])
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

In [ ]:
#Célula 6: Treinamento do Modelo Black-Box

model = XGBClassifier(
    n_estimators=100,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train_enc, y_train)

print("Modelo treinado com sucesso!")

In [ ]:
#Célula 7: Configuração do Shapash (A Camada de XAI)

#Dicionário do Dashboard
features_dict = {
    'Temp_Ar': 'Temperatura do Ambiente',
    'Temp_Processo': 'Temperatura do Processo',
    'Rotacao_rpm': 'Velocidade de Rotação',
    'Torque_Nm': 'Torque do Motor',
    'Desgaste_Ferramenta_min': 'Tempo de Uso da Ferramenta',
    'Tipo_Equipamento': 'Categoria da Máquina'
}

#Instancia do SmartExplainer
xpl = SmartExplainer(
    model=model,
    preprocessing=encoder, # Passamos o encoder para ele reverter os números para texto (L, M, H) no gráfico
    features_dict=features_dict
)

In [ ]:
#Célula 8: Compilação das Explicações

xpl.compile(
    x=X_test_enc,
    y_target=y_test # Opcional: permite comparar Previsão vs Realidade no dashboard
)

In [ ]:
#Célula 9: Lançamento do WebApp

app = xpl.run_app(title_story="Monitor de Manutenção Preditiva - Indústria 5.0")